In [1]:
from bdsg.bdsg import HashGraph
from bdsg.bdsg import SnarlDistanceIndex
from bdsg.bdsg import PackedGraph

In [2]:
import sys
sys.path.append('./assembler')
from anchor import SnarlAnchor

#### STEP 1: IMPORT THE DATA 
1. The graph in PackedGraph format (using ```vg convert -p``` ) 
2. The distance index for the snarl/chain tree

In [3]:
graph_path: str = 'small_test/graph.vg'#'small_test/chr20_small_idx.vg'
index_path: str = 'small_test/graph.dist'#'small_test/chr20_small_idx.dist'

In [ ]:
graph = PackedGraph()
graph.deserialize(graph_path) # .vg


idx = SnarlDistanceIndex()
idx.deserialize(index_path) # .dist

##### TEST 1: CHECK HOW THE TREE STRUCTURE WORKS. 
1. In theory the root of the tree is going to be a chain.
2. I iterate over its children and if I find a snarl, I check if it is a leaf snarl.
3. Each snarl has as a child a chain. If the children of the chain are not snarls, it is a leaf snarl.
4. Else if it has other snarls, I keep iterating until I find the leaf snarl.


Which functions do I need?
1. from a chain, traverse its children. 
2. If find a snarl. Iterate over its children (chains(s)).
3. If the childern of the chain of the snarl are all nodes, append the snarl to the list;
4. Else go to 2.

In [ ]:
root_handle = idx.get_root()

In [ ]:
leaf_snarls = [] 
contains_child_snarls = False
num_nodes = 0

def check_for_snarl(child_net_handle):
    global contains_child_snarls
    global num_nodes
    if idx.is_snarl(child_net_handle):
        contains_child_snarls = True
    elif idx.is_node(child_net_handle):
        num_nodes += 1
    return True

# THIS FUNCTION TAKES A SNARL. FOR EACH CHILD (CHAIN) OF THE SNARL, CHECK THEIR CHILDREN. 
# IF NO ONE HAS A SNARL, THE SNARL IS A LEAF
def snarl_iteratee(handle):
    global contains_child_snarls
    contains_child_snarls = False
    snarl_children = []
    idx.for_each_child(handle, lambda y: snarl_children.append(y) or True) 
    
    num_nodes = 0
    for s_c in snarl_children:
        idx.for_each_child(s_c, check_for_snarl)
    
    if ((not contains_child_snarls) and (num_nodes < 10)):
        leaf_snarls.append(handle)
    return True
idx.traverse_decomposition(snarl_iteratee, lambda x: True, lambda y: True)
# THIS FUNCTION TAKES A SNARL. IF IT HAS MORE THAN 1 CHILD, IT MEANS IT IS A LEAF
# THIS FROM THE INTUITION THAT AN INTERNAL SNARLS HAS AS 1 CHILD THAT IS A CHAIN OF NODES AND SNARL(S)
# NOT SURE IT IS TRUE ALWAYS
def snarl_inf(handle):
    snarl_children = []
    idx.for_each_child(handle, lambda y: snarl_children.append(y) or True)
    
    if len(snarl_children) > 1:
        leaf_snarls_inf.append(handle)
    return True


leaf_snarls_inf = [] 
idx.traverse_decomposition(snarl_inf, lambda x: True, lambda y: True)

print('printing out')
for el in leaf_snarls:
    print(idx.net_handle_as_string(el))
    start_bound = idx.get_start_bound(el)
    end_bound = idx.get_end_bound(el)

    # Inspect the orientations
    print(f"Start Bound ID: {graph.get_id(idx.get_handle(start_bound, graph))}, is_reverse: {graph.get_is_reverse(idx.get_handle(start_bound, graph))}")
    print(f"End Bound ID: {graph.get_id(idx.get_handle(end_bound, graph))}, is_reverse: {graph.get_is_reverse(idx.get_handle(end_bound, graph))}")
    
for el in leaf_snarls_inf:
    print(idx.net_handle_as_string(el))

#### STEP 2: GENERATE THE ANCHOR DICTIONARY
1. Traverse the SNARL TREE, using ```index.traverse_decomposition```
2. When detecting a LEAF SNARL, PASS IT TO THE SNARL CONSTRUCTION
3.  When detecting a LEAF SNARL:
    1. The snarl has to contain less than X = 10 elements; X is a parameter. 
    2. The number of paths passing by the snarl has to be > MIN and < MAX. Both parameters 
    3. For each path in the snarl, 

In [ ]:
anchoring = SnarlAnchor(10)

In [ ]:
anchoring.build_graph(graph_path, index_path)

In [ ]:
leaf_snarl_net_handles: list = anchoring.get_leaf_snarls()

In [ ]:
leaf_snarl_net_handles

In [ ]:
anchoring.print_tree_structure()

In [ ]:
one_snarl = leaf_snarl_net_handles[0]
# get the start and end bounds (net handles)
start_bound = idx.get_start_bound(one_snarl)
end_bound = idx.get_end_bound(one_snarl)
print(f"Start bound is {idx.net_handle_as_string(start_bound)}")

# get node handles and orientations 
start_node_id = idx.node_id(start_bound)
start_node_handle = idx.get_handle(start_bound,graph)
end_node_id = idx.node_id(end_bound)
end_node_handle = idx.get_handle(end_bound,graph)

# check if they are in reverse orientation
start_is_reverse = graph.get_is_reverse(start_node_handle)
end_is_reverse = graph.get_is_reverse(end_node_handle)

# print info
start_direction = "reverse" if start_is_reverse else "forward"
end_direction = "reverse" if end_is_reverse else "forward"

print(f"Start node of snarl: {start_node_id}, Direction: {start_direction}")
print(f"End node of snarl: {end_node_id}, Direction: {end_direction}")
# print(start_node_id, end_node_id)

In [ ]:
graph_id = graph.get_id(start_node_handle)

In [ ]:
graph_id

In [ ]:
steps_on_start_node = []
graph.for_each_step_on_handle(start_node_handle, lambda y: steps_on_start_node.append(y) or True)

In [ ]:
steps_on_start_node

In [ ]:
path_handles = []
path_names = []
for step in steps_on_start_node:
    path_handle = graph.get_path_handle_of_step(step)
    path_handles.append(path_handle)
    path_name = graph.get_path_name(path_handle)
    path_names.append(path_name)


In [ ]:
path_names

In [ ]:
path_1_handle = path_handles[0]
path_1_name = path_names[0]

sentinels = [graph.get_id(end_node_handle), graph.get_id(start_node_handle)] \
    if start_is_reverse else [graph.get_id(start_node_handle), graph.get_id(end_node_handle)]
print(sentinels[0],sentinels[1])
print(f"Walking on path {path_1_name}")

# Step 3: Traverse all the steps on the path using for_each_step_in_path
def traverse_step(step_handle):
    # For each step, get the corresponding node handle (handle_t)
    node_handle = graph.get_handle_of_step(step_handle)
    # Get information about this node
    node_id = graph.get_id(node_handle)
    print(f"visiting {node_id}", end=": ")
    global keep
    if not keep and node_id not in sentinels:
        print('skipped.')
        return True
    
    node_sequence = graph.get_sequence(node_handle)
    is_reversed = graph.get_is_reverse(node_handle)
    traversal.append((node_id,not(is_reversed)))
    direction = "reverse" if is_reversed else "forward"
    print('visited.')
    
    # Output the node details and whether it's traversed forward or reverse
    print(f"Node {node_id}, Sequence: {node_sequence}, Traversed in {direction} direction")
    if keep and node_id in sentinels:
        print('ending visit.')
        keep = False
        return False
    
    keep = True
    return True  # Continue traversing



# Traverse the entire path and process each step
traversals = []
traversal = []
keep = False

for path_h in path_handles:
    graph.for_each_step_in_path(path_h, traverse_step)
    traversals.append(traversal)
    traversal = []

In [ ]:
def print_traversal(trv):
    for node in trv:
        direction = ">" if node[1] == True else "<"
        print(f"{direction}{node[0]}",end="")
    print()

for pt,trv in zip(path_names,traversals):
    print(f"{pt}: ",end="")
    print_traversal(trv)

In [ ]:
def get_anchor_id(traversal):
    anchor = traversal[len(traversal)//2][0]
    return anchor
        

In [ ]:
for trv in traversals:
    print(get_anchor_id(trv), end=" ")
    print_traversal(trv)

In [ ]:
def is_equal_path(path_1,path_2):
    #if different length, false
    if len(path_1) != len(path_2):
        return False
    
    #start at 0
    pos_path_1 = 0 if path_1[0][1] else len(path_1) - 1
    pos_path_2 = 0 if path_2[0][1] else len(path_2) - 1
    for i in range(len(path_1)):
        start1,orientation1 = path_1[pos_path_1]
        start2,orientation2 = path_2[pos_path_2]
        if start1 == start2 and orientation1 == orientation2:
            continue
        else: return False
    return True

In [ ]:
paths_dict = dict()
for trv in traversals:
    anchor = get_anchor_id(trv)
    print(f"{anchor}:", end=" ")
    print_traversal(trv)
    
    if anchor not in paths_dict:
        paths_dict[anchor] = [trv]
    else:
        possible_paths = paths_dict[anchor]
        insert = True
        for pts in possible_paths:
            if is_equal_path(trv,pts):
                insert = False
                print("not accepted")
                break
        if insert:
            paths_dict[anchor].append(trv)
        
        
print(paths_dict)

In [ ]:
snarl_traversals = anchoring.get_paths_traversing_snarl(one_snarl)

In [ ]:
snarl_traversals

In [ ]:
for trvrsl in snarl_traversals:
    anchor_length = 0
    for node_handle in trvrsl:
        anchor_length += graph.get_length(node_handle)
        print(f'tmp_al: {anchor_length}', end = " ")
    anchor_length -= (graph.get_length(trvrsl[0]) + graph.get_length(trvrsl[len(trvrsl) - 1]))// 2
    print(f'final_al: {anchor_length}')

In [ ]:
for trvrsl in snarl_traversals:
    anchoring.print_traversal(trvrsl)

In [ ]:


tr1,tr2 = snarl_traversals[2:4]

for i in range(len(tr1)):
    node_h_tr1 = tr1[i]
    node_h_tr2 = tr2[i]
    direction = "<" if graph.get_is_reverse(node_h_tr1) == True else ">"
    print(f"TR1: {direction}{graph.get_id(node_h_tr1)}",end=" - ")
    direction = "<" if graph.get_is_reverse(node_h_tr2) == True else ">"
    print(f"TR2: {direction}{graph.get_id(node_h_tr2)}",end=" - ")
    if tr1[i] == tr2[i]:
        print("equals")
    else:
        print("not_equals")
    

In [ ]:
anchoring.fill_anchor_sentinel_table(one_snarl)

In [ ]:
anchoring.sentinel_to_anchor

##### TESTING ON A 500 bases real window from chromosome 20 (5000 bps from start of chm13)
1. Load indexes
2. generate snarls list
3. generate anchor dictionary

In [4]:
anchoring = SnarlAnchor(10)
anchoring.build_graph(graph_path, index_path)

In [5]:
anchoring.print_tree_structure()

Chain: chain 2367365rev->2367337revtraversing start->end
Node: node 2367365rev
Snarl: simple snarl 2367365rev->2367362revtraversing start->end
Chain: node 2367364fd pretending to be a chain in a simple snarl
Node: node 2367364fd
Chain: node 2367363fd pretending to be a chain in a simple snarl
Node: node 2367363fd
Node: node 2367362rev
Snarl: simple snarl 2367362rev->2367359revtraversing start->end
Chain: node 2367360fd pretending to be a chain in a simple snarl
Node: node 2367360fd
Chain: node 2367361fd pretending to be a chain in a simple snarl
Node: node 2367361fd
Node: node 2367359rev
Node: node 2367358rev
Snarl: simple snarl 2367358rev->2367356revtraversing start->end
Chain: node 2367357fd pretending to be a chain in a simple snarl
Node: node 2367357fd
Node: node 2367356rev
Snarl: simple snarl 2367356rev->2367353revtraversing start->end
Chain: node 2367354fd pretending to be a chain in a simple snarl
Node: node 2367354fd
Chain: node 2367355fd pretending to be a chain in a simple sn

In [6]:
leaf_snarl_net_handles: list = anchoring.get_leaf_snarls()

Visiting simple snarl 2367365rev->2367362revtraversing start->end
Children: node 2367364fd has #nodes: 1 and has_snarls is: False
Children: node 2367363fd has #nodes: 2 and has_snarls is: False
Visiting simple snarl 2367362rev->2367359revtraversing start->end
Children: node 2367360fd has #nodes: 1 and has_snarls is: False
Children: node 2367361fd has #nodes: 2 and has_snarls is: False
Visiting simple snarl 2367358rev->2367356revtraversing start->end
Children: node 2367357fd has #nodes: 1 and has_snarls is: False
Visiting simple snarl 2367356rev->2367353revtraversing start->end
Children: node 2367354fd has #nodes: 1 and has_snarls is: False
Children: node 2367355fd has #nodes: 2 and has_snarls is: False
Visiting simple snarl 2367353rev->2367350revtraversing start->end
Children: node 2367351fd has #nodes: 1 and has_snarls is: False
Children: node 2367352fd has #nodes: 2 and has_snarls is: False
Visiting simple snarl 2367350rev->2367347revtraversing start->end
Children: node 2367348fd has

In [ ]:
one_snarl = leaf_snarl_net_handles[1]

In [ ]:
one_snarl

In [ ]:
snarl_traversals = anchoring.get_paths_traversing_snarl(one_snarl)

In [7]:
anchoring.fill_anchor_sentinel_table()

Obtaining steps on start node for snarl <bdsg.handlegraph.net_handle_t object at 0x7f72f012f530>... Done.
Obtaining path handles from steps... Done.
Obtaining path in snarls... anchor of length 21 is too short
anchor of length 21 is too short
anchor of length 21 is too short
anchor of length 21 is too short
anchor of length 21 is too short
anchor of length 21 is too short
anchor of length 21 is too short
anchor of length 21 is too short
anchor of length 21 is too short
anchor of length 21 is too short
Done.
Obtaining steps on start node for snarl <bdsg.handlegraph.net_handle_t object at 0x7f72f012d170>... Done.
Obtaining path handles from steps... Done.
Obtaining path in snarls... anchor inserted
anchor inserted
anchor inserted
anchor inserted
anchor inserted
anchor inserted
anchor inserted
anchor inserted
anchor inserted
anchor inserted
Done.
Obtaining steps on start node for snarl <bdsg.handlegraph.net_handle_t object at 0x7f72f012f7f0>... Done.
Obtaining path handles from steps... D

In [8]:
anchoring.print_anchors_from_dict()

Sentinel: 2367360 ; Anchor : <2367362<2367360<2367359 ; Bandage : ,2367362,2367360,2367359
Sentinel: 2367357 ; Anchor : <2367358<2367357<2367356 ; Bandage : ,2367358,2367357,2367356
Sentinel: 2367354 ; Anchor : <2367356<2367354<2367353 ; Bandage : ,2367356,2367354,2367353
Sentinel: 2367355 ; Anchor : >2367353>2367355>2367356 ; Bandage : ,2367353,2367355,2367356
Sentinel: 2367351 ; Anchor : <2367353<2367351<2367350 ; Bandage : ,2367353,2367351,2367350
Sentinel: 2367352 ; Anchor : >2367350>2367352>2367353 ; Bandage : ,2367350,2367352,2367353
Sentinel: 2367349 ; Anchor : <2367350<2367349<2367347 ; Bandage : ,2367350,2367349,2367347
Sentinel: 2367348 ; Anchor : >2367347>2367348>2367350 ; Bandage : ,2367347,2367348,2367350
Sentinel: 2367345 ; Anchor : >2367343>2367345>2367346 ; Bandage : ,2367343,2367345,2367346
Sentinel: 2367344 ; Anchor : >2367343>2367344>2367346 ; Bandage : ,2367343,2367344,2367346
Sentinel: 2367343 ; Anchor : >2367341>2367343 ; Bandage : ,2367341,2367343
Sentinel: 23673